In [1]:
import findspark, pyspark

from pyspark.sql               import SparkSession
from pyspark.sql.functions     import isnan, when, count, col
from pyspark.ml                import Pipeline
from pyspark.ml.feature        import RFormula, VectorAssembler, Binarizer, Normalizer, OneHotEncoder
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation     import BinaryClassificationEvaluator
from pyspark.ml.evaluation     import MulticlassClassificationEvaluator
from pyspark.ml.feature        import StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier

findspark.init()
spark = SparkSession.builder.appName("pipeline").getOrCreate()

In [7]:
cancerDF = spark.read.csv("Breast_cancer_data.csv", header=True)
cancerDF.show(2,truncate=False)

+-----------+------------+--------------+---------+---------------+---------+
|mean_radius|mean_texture|mean_perimeter|mean_area|mean_smoothness|diagnosis|
+-----------+------------+--------------+---------+---------------+---------+
|17.99      |10.38       |122.8         |1001.0   |0.1184         |0        |
|20.57      |17.77       |132.9         |1326.0   |0.08474        |0        |
+-----------+------------+--------------+---------+---------------+---------+
only showing top 2 rows



In [8]:
print(cancerDF.count())

569


In [9]:
cancerDF.select([count(when(isnan(c), c)).alias(c) for c in cancerDF.columns]).show()

+-----------+------------+--------------+---------+---------------+---------+
|mean_radius|mean_texture|mean_perimeter|mean_area|mean_smoothness|diagnosis|
+-----------+------------+--------------+---------+---------------+---------+
|          0|           0|             0|        0|              0|        0|
+-----------+------------+--------------+---------+---------------+---------+



In [10]:
Rformula = RFormula(formula="diagnosis ~ mean_radius + mean_texture + mean_perimeter + mean_area + mean_smoothness", 
                    featuresCol="independente", labelCol="dependente")

cancerDF = Rformula.fit(cancerDF).transform(cancerDF)
cancerDF.show()

+-----------+------------+--------------+---------+---------------+---------+--------------------+----------+
|mean_radius|mean_texture|mean_perimeter|mean_area|mean_smoothness|diagnosis|        independente|dependente|
+-----------+------------+--------------+---------+---------------+---------+--------------------+----------+
|      17.99|       10.38|         122.8|   1001.0|         0.1184|        0|(2465,[87,536,104...|       1.0|
|      20.57|       17.77|         132.9|   1326.0|        0.08474|        0|(2465,[383,690,95...|       1.0|
|      19.69|       21.25|         130.0|   1203.0|         0.1096|        0|(2465,[364,517,94...|       1.0|
|      11.42|       20.38|         77.58|    386.1|         0.1425|        0|(2465,[135,774,12...|       1.0|
|      20.29|       14.34|         135.1|   1297.0|         0.1003|        0|(2465,[375,592,10...|       1.0|
|      12.45|        15.7|         82.57|    477.1|         0.1278|        0|(2465,[46,456,131...|       1.0|
|      18.

In [12]:
cancerTreino, cancerTeste = cancerDF.randomSplit([0.7,0.3])
print(cancerTreino.count())
print(cancerTeste.count())

385
184


In [ ]:
# Regressão Logistica

In [13]:
logistic = LogisticRegression(featuresCol="independente", labelCol="dependente", maxIter=1000, regParam=0.08)
modelo = logistic.fit(cancerTreino)

In [14]:
predicao = modelo.transform(cancerTeste)

In [15]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(predicao)
print(acuracia)

0.6304347826086957


In [ ]:
# Randon Forest

In [16]:
randomforest = RandomForestClassifier(labelCol="dependente", featuresCol="independente", numTrees=40)
modelo = randomforest.fit(cancerTreino)

In [17]:
predicao = modelo.transform(cancerTeste)

In [18]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(predicao)
print(acuracia)

0.6358695652173914


In [ ]:
# Gradient-Boosted Trees (GBTs)

In [20]:
gbt = GBTClassifier(labelCol="dependente", featuresCol="independente")
modelo = gbt.fit(cancerTreino)

In [21]:
predicao = modelo.transform(cancerTeste)

In [22]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(predicao)
print(acuracia)

0.6467391304347826


In [ ]:
# MultiLayer Perceptron

In [27]:
print(len(cancerDF.columns))

8


In [28]:
mlp = MultilayerPerceptronClassifier(maxIter=1000, layers=[8,5,2], featuresCol="independente", labelCol="dependente")
modelo = mlp.fit(cancerTreino)

In [29]:
predicao = modelo.transform(cancerTeste)

In [30]:
performance = MulticlassClassificationEvaluator(labelCol="dependente", predictionCol="prediction", metricName="accuracy")
acuracia = performance.evaluate(predicao)
print(acuracia)

Py4JJavaError: An error occurred while calling o892.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 465.0 failed 1 times, most recent failure: Lost task 0.0 in stage 465.0 (TID 453) (DESKTOP-KETEVB0.mshome.net executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (ProbabilisticClassificationModel$$Lambda$4298/0x00000001019ec040: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:217)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowI